In [16]:
import numpy as np
from py3gpp import *

In [17]:
A = 10000           # Transport block length
rate = 449 / 1024   # Target code rate
rv = 0              # Redundancy version, 0-3
modulation = '16QAM'
nlayers = 1

cbs_info = nrDLSCHInfo(A, rate)
print(cbs_info)

{'CRC': '24A', 'L': 24, 'BGN': 1, 'C': 2, 'Lcb': 24, 'F': 244, 'Zc': 240, 'K': 5280, 'N': 15840}


In [18]:
in_ = np.random.randint(0, 2, (A, 1))

tb_in = nrCRCEncode(in_, cbs_info['CRC'])

cbs_in = nrCodeBlockSegmentLDPC(tb_in, cbs_info['BGN'])

enc = nrLDPCEncode(cbs_in, cbs_info['BGN'])

outlen = np.ceil(A/rate)
ch_in = nrRateMatchLDPC(enc, outlen, rv, modulation, nlayers)

In [19]:
# channel
if True:
    nid = 1
    pdsch_bits = nrPDSCH(np.expand_dims(ch_in, 0), [modulation], 1, nid, 0)
    ch_out = nrPDSCHDecode(pdsch_bits, [modulation], nid, 0)[0]
else:
    ch_out = 1-2*(ch_in)

In [20]:
raterec = nrRateRecoverLDPC(ch_out, A, rate, rv, modulation, nlayers)

dec_bits, iters = nrLDPCDecode(raterec, cbs_info['BGN'], 25)
print(f'decoding finished after {iters} iterations')

blk, blk_err = nrCodeBlockDesegmentLDPC(dec_bits, cbs_info['BGN'], A + cbs_info['L'])
print(f'code block CRC: {int(blk_err)}')

out, tb_err = nrCRCDecode(blk, cbs_info['CRC'])
print(f'transport block CRC: {tb_err}')

print(f'output equals input: {np.array_equal(in_, out)}')

decoding finished after 25 iterations
code block CRC: 0
transport block CRC: [0]
output equals input: True
